# Toxic Comment

## Chargement des données

In [1]:
import data.read_data as read
import data.transform_data as transform
from numpy import concatenate
from math import ceil


In [2]:
train_csv_file_name = "res/train.csv"
test_csv_file_name = "res/test.csv"
test_label_csv_file_name = "res/test_labels.csv"
submission_csv_file_name = "res/sample_submission.csv"


In [3]:
df_train = read.load_train_csv(train_csv_file_name)


In [4]:
print(df_train)


                      id                                       comment_text  \
0       0000997932d777bf  Explanation\nWhy the edits made under my usern...   
1       000103f0d9cfb60f  D'aww! He matches this background colour I'm s...   
2       000113f07ec002fd  Hey man, I'm really not trying to edit war. It...   
3       0001b41b1c6bb37e  "\nMore\nI can't make any real suggestions on ...   
4       0001d958c54c6e35  You, sir, are my hero. Any chance you remember...   
5       00025465d4725e87  "\n\nCongratulations from me as well, use the ...   
6       0002bcb3da6cb337       COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK   
7       00031b1e95af7921  Your vandalism to the Matt Shirvington article...   
8       00037261f536c51d  Sorry if the word 'nonsense' was offensive to ...   
9       00040093b2687caa  alignment on this subject and which are contra...   
10      0005300084f90edc  "\nFair use rationale for Image:Wonju.jpg\n\nT...   
11      00054a5e18b50dd4  bbq \n\nbe a man and lets 

In [5]:
for col in df_train.iloc[:, 2:]:
    print(col, ":", df_train[col].sum(), "/", len(df_train))


toxic : 15294 / 159571
severe_toxic : 1595 / 159571
obscene : 8449 / 159571
threat : 478 / 159571
insult : 7877 / 159571
identity_hate : 1405 / 159571


In [6]:
print(df_train.dtypes)


id               object
comment_text     object
toxic             int64
severe_toxic      int64
obscene           int64
threat            int64
insult            int64
identity_hate     int64
dtype: object


In [7]:
limit_train = 120000


In [8]:
X_train = df_train.iloc[:limit_train, :2].values
Y_train = df_train.iloc[:limit_train, 2:].values

X_test = df_train.iloc[limit_train:, :2].values
Y_test = df_train.iloc[limit_train:, 2:].values

X_train = transform.split_comment(X_train)
X_test = transform.split_comment(X_test)

count, vocab = transform.mk_vocab(X_train + X_test)


In [9]:
print(len(vocab))


25385


In [10]:
print(X_train[0])


('0000997932d777bf', array(['explanation', 'why', 'the', 'edits', 'made', 'under', 'my',
       'username', 'hardcore', 'metallica', 'fan', 'were', 'reverted',
       'they', 'weren', 't', 'vandalisms', 'just', 'closure', 'on',
       'some', 'gas', 'after', 'i', 'voted', 'at', 'new', 'york', 'dolls',
       'fac', 'and', 'please', 'don', 't', 'remove', 'the', 'template',
       'from', 'the', 'talk', 'page', 'since', 'i', 'm', 'retired', 'now',
       '89', '205', '38', '27'], dtype='<U11'))


In [11]:
ident_train, X_train = transform.pass_data_to_word_idx(X_train, vocab, count)
ident_test, X_test = transform.pass_data_to_word_idx(X_test, vocab, count)


In [12]:
max_len = transform.get_max_len_sent(concatenate((X_train, X_test), axis=0))
print(max_len)
print(concatenate((X_train, X_test), axis=0))
print(X_train)


1400
[array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 16,
       34,  3, 35, 36,  3, 37, 38, 39, 24, 40, 41, 42, 43, 44, 45, 46])
 array([47, 48, 49, 50, 51, 52, 53, 24, 40, 54, 55, 56, 57, 37, 58, 59, 60,
       61, 62, 63])
 array([64, 65, 24, 40, 66, 67, 68, 69, 70, 71, 72, 73, 18, 74, 51, 75, 76,
       77, 78, 79, 80, 31, 81, 69, 82, 83,  4, 84, 85,  7, 37, 38, 49, 86,
       69, 87, 88, 89,  3, 90, 91,  3, 92, 93])
 ...
 array([ 8954,  2380,   128,    92,   176,   136,  6633,  2059, 15974,
        3645])
 array([  31,   72, 1415,  384,   72,  185,  893,  133,  294, 1130,   20,
          3,  279,   69,  134,    3,  132,  616,  275,   42,   74,   24,
        383,   26,   72])
 array([  31,   24,   66,   33,   16,  112,  133,  577,   24,  446,  506,
         31,    7, 2501,  185,  529, 1107, 1836,  162, 1839,   85, 1771,
       1062,  133,  134,  529, 2580,  242, 1836,   84,   85, 6725,

In [13]:
mean = 0
total = 0
for s in concatenate((X_train, X_test), axis=0):
    total += 1
    mean += s.shape[0]
print(mean / total)


67.03647905947822


In [14]:
X_train = transform.pad_sentences(X_train, max_len, vocab[transform.padding])
X_test = transform.pad_sentences(X_test, max_len, vocab[transform.padding])


In [15]:
print(X_train[0:10])


[[  1.   2.   3. ...   0.   0.   0.]
 [ 47.  48.  49. ...   0.   0.   0.]
 [ 64.  65.  24. ...   0.   0.   0.]
 ...
 [173. 174.  69. ...   0.   0.   0.]
 [182. 101.   3. ...   0.   0.   0.]
 [218.  20.  51. ...   0.   0.   0.]]


In [16]:
print(X_train.shape)


(120000, 1400)


In [17]:
print(Y_test[0])
print(Y_test.shape)


[0 0 0 0 0 0]
(39571, 6)


## Apprentissage

In [18]:
from model.conv_model import ConvModel, ConvModel2, ConvModel3
from model.recurrent_model import RecurrentModel
import torch as th
import torch.nn as nn
from utils.cuda import use_cuda
from tqdm import tqdm
from torchnet.meter import AUCMeter
import sys


In [19]:
th.cuda.is_available()


True

In [20]:
print(transform.padding, vocab[transform.padding])
print(len(vocab), max_len, vocab[transform.padding])


<pad> 0
25385 1400 0


In [21]:
model = RecurrentModel(len(vocab), 32, max_len, vocab[transform.padding])
# model = ConvModel(len(vocab), max_len, vocab[transform.padding])
loss_fn = nn.BCELoss()

if use_cuda():
    model.cuda()
    loss_fn.cuda()

opt = th.optim.Adagrad(model.parameters(), lr=5e-4)

nb_epoch = 10

batch_size = 32
nb_batch = ceil(X_train.shape[0] / batch_size)

auc_meters = {0:AUCMeter(), 1:AUCMeter(), 2:AUCMeter(), 3:AUCMeter(), 4:AUCMeter(), 5:AUCMeter()}

losses = []
mean_aucs = []
label_aucs = []

for e in range(nb_epoch):
    model.train()
        
    sum_loss = 0
    nb_sent = 0
    
    for i in tqdm(range(nb_batch)):
        i_min = batch_size * i
        i_max = batch_size * (i + 1) if batch_size * (i + 1) < X_train.shape[0] else X_train.shape[0]
                
        x, y = th.Tensor(X_train[i_min:i_max]).long(), th.Tensor(Y_train[i_min:i_max])
        
        if use_cuda():
            x, y = x.cuda(), y.cuda()
        
        out = model(x)
        loss = loss_fn(out, y)
        loss.backward()
        opt.step()
        
        sum_loss += loss.item()
        nb_sent += 1
    
    sum_loss /= nb_sent
    
    losses.append(sum_loss)
    
    print("Epoch %d : loss = %f" % (e, sum_loss))
    
    model.eval()
    
    for i in range(6):
        auc_meters[i].reset()
    
    x_test = th.Tensor(X_test).long()
    y_test = th.Tensor(Y_test)
    
    batch_size_test = 32
    nb_batch_test = ceil(x_test.size(0) / batch_size_test)
    
    for i in tqdm(range(nb_batch_test)):
        i_min = batch_size_test * i
        i_max = batch_size_test * (i + 1) if batch_size_test * (i + 1) < x_test.size(0) else x_test.size(0)
        
        x, y = x_test[i_min:i_max], y_test[i_min:i_max]
        
        if use_cuda():
            x, y = x.cuda(), y.cuda()
        
        out_test = model(x)
        
        for l in range(6):
            auc_meters[l].add(out_test[:, l].detach(), y[:, l].detach())
        
    roc_auc_scores = {
        "toxic": auc_meters[0].value()[0],
        "severe_toxic": auc_meters[1].value()[0],
        "obscene": auc_meters[2].value()[0],
        "threat": auc_meters[3].value()[0],
        "insult": auc_meters[4].value()[0],
        "identity_hate": auc_meters[5].value()[0]
    }
    
    print(roc_auc_scores)
    mean = sum([v for _, v in roc_auc_scores.items()]) / 6
    mean_aucs.append(mean)
    label_aucs.append(roc_auc_scores)
    print("Mean ROC AUC : %f" % mean)
    sys.stdout.flush()


  0%|          | 0/3750 [00:00<?, ?it/s]

  0%|          | 2/3750 [00:00<03:09, 19.78it/s]

  0%|          | 5/3750 [00:00<03:03, 20.40it/s]

  0%|          | 8/3750 [00:00<02:56, 21.14it/s]

  0%|          | 11/3750 [00:00<02:52, 21.70it/s]

...

 99%|█████████▉| 1225/1237 [00:28<00:00, 42.05it/s]

 99%|█████████▉| 1230/1237 [00:28<00:00, 42.08it/s]

100%|█████████▉| 1235/1237 [00:28<00:00, 42.31it/s]

100%|██████████| 1237/1237 [00:28<00:00, 43.21it/s]

{'toxic': 0.8332103438768719, 'severe_toxic': 0.9213002406536681, 'obscene': 0.8509921254494189, 'threat': 0.8778005725975735, 'insult': 0.8638912354194928, 'identity_hate': 0.8560913710833357}
Mean ROC AUC : 0.867214


In [22]:
print(model)


RecurrentModel(
  (emb): Embedding(25385, 16, padding_idx=0)
  (lstm): LSTM(16, 32, batch_first=True)
  (maxpool): MaxPool1d(kernel_size=1400, stride=1400, padding=0, dilation=1, ceil_mode=False)
  (pred): Sequential(
    (0): Linear(in_features=32, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=6, bias=True)
    (3): Sigmoid()
  )
)


## Sauvegarde du modèle, vocabulaire etc.

In [23]:
import pickle


In [24]:
pickle.dump(losses, open("losses_rnn.pkl", "wb"))
pickle.dump(mean_aucs, open("mean_auc.pkl", "wb"))
pickle.dump(label_aucs, open("label_aucs.pkl", "wb"))


In [24]:
saved_model = model.cpu()
saved_loss = loss_fn.cpu()


In [25]:
import datetime
dt = datetime.datetime.now()
saved_identity = "_conv1_120k_" + dt.strftime("%Hh_%Mm_%Ss_%d_%m_%Y")


In [26]:
pickle.dump(model, open("saved/conv_6/model" + saved_identity + ".p", "wb"))
pickle.dump(loss_fn, open("saved/conv_6/loss_fn" + saved_identity + ".p", "wb"))
pickle.dump(opt, open("saved/conv_6/opt" + saved_identity + ".p", "wb"))
pickle.dump(vocab, open("saved/conv_6/vocab" + saved_identity + ".p", "wb"))
pickle.dump(count, open("saved/conv_6/count" + saved_identity + ".p", "wb"))


In [25]:
import matplotlib.pyplot as plt

In [26]:
plt.figure()
plt.plot(losses, label="Loss value")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("BCELoss with RecurrentModel")
plt.legend()
plt.show()

In [31]:
plt.figure()
plt.plot(mean_aucs, label="AUROC mean value", c="r")
plt.xlabel("Epoch")
plt.ylabel("AUROC mean")
plt.title("AUROC mean with RecurrentModel")
plt.legend()
plt.show()


In [32]:
auroc = []
labels = {
        0:"toxic",
        1:"severe_toxic",
        2:"obscene",
        3:"threat",
        4:"insult",
        5:"identity_hate"}

for l in range(6):
    tmp = []
    for i in range(len(label_aucs)):
        tmp.append(label_aucs[i][labels[l]])
    auroc.append(tmp)


In [36]:
plt.figure()
for l in range(6):
    plt.plot(auroc[l], label="AUROC value %s" % labels[l])
plt.xlabel("Epoch")
plt.ylabel("AUROC value")
plt.title("AUROC per labels with RecurrentModel")
plt.legend()
plt.show()